In [ ]:
!pip install -q -U transformers==4.37.2
!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 33.0 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
from transformers import pipeline

model_id = "llava-hf/llava-1.5-13b-hf"

pipe = pipeline("image-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

In [ ]:
from PIL import Image, ImageDraw
import os
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

#folders
folder_path_images = '/content/drive/MyDrive/augumented_q-1_images/images'
folder_path_annotations = '/content/drive/MyDrive/augumented_q-1_images/annotations'

# List all image files
image_paths = [os.path.join(folder_path_images, f) for f in os.listdir(folder_path_images) if f.endswith('.jpg')]

questions = [
    "Question 1: <image> Is he wearing hard hat? \nASSISTANT:",
]

predictions = []
ground_truths = []
files = []

# Processing each image and extracting predictions
for image_path in image_paths:
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # Corresponding annotation file
    base_filename = os.path.splitext(os.path.basename(image_path))[0]
    annotation_file = f"{folder_path_annotations}/{base_filename}.txt"

    #the ground truth annotation
    with open(annotation_file, 'r') as file:
        ground_truth = file.read().strip().lower()
        ground_truths.append(ground_truth)
        files.append(base_filename)  # Storing the base filename for tracing

    for question in questions:
        outputs = pipe(image, prompt=question, generate_kwargs={"max_new_tokens": 200})
        generated_text = outputs[0]["generated_text"]
        assistant_response = generated_text.split("\nASSISTANT:")[1].strip()

        first_word = assistant_response.split()[0].replace(',', '').lower() if assistant_response else "no response"
        predictions.append(first_word)


        print(f"Image Path: {image_path}")
        print(f"Prediction: {first_word}")
        print(f"Ground Truth: {ground_truth}")
        print("-------------------------------")

# Calculating the metrics
accuracy = accuracy_score(ground_truths, predictions)
precision = precision_score(ground_truths, predictions, pos_label='yes')
recall = recall_score(ground_truths, predictions, pos_label='yes')
f1 = f1_score(ground_truths, predictions, pos_label='yes')

# Display aggregated metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")